In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import thundersvm
import warnings

In [3]:
df = pd.read_csv("iot23_combined.csv", nrows=800000)

In [4]:
df

,Unnamed: 0,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
1,1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
2,2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
3,3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,Benign,...,False,False,False,True,False,False,False,False,False,False
4,4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,Benign,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,59971,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,...,False,False,False,True,False,False,False,False,False,False
799996,59972,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,...,False,False,False,True,False,False,False,False,False,False
799997,59973,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,...,False,False,False,True,False,False,False,False,False,False
799998,59974,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,...,False,False,False,True,False,False,False,False,False,False


In [5]:
del df['Unnamed: 0']

In [6]:
df.shape

(800000, 25)

In [7]:
df['label'].value_counts()

label
PartOfAHorizontalPortScan    369546
Okiru                        209609
Benign                       162421
DDoS                          39353
C&C                           15009
Attack                         3814
C&C-HeartBeat                   210
C&C-Torii                        30
C&C-FileDownload                  8
Name: count, dtype: int64

In [8]:
df.columns = ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']

In [9]:
for col in df.columns: 
    print(col) 

duration
orig_bytes
resp_bytes
missed_bytes
orig_pkts
orig_ip_bytes
resp_pkts
resp_ip_bytes
label
proto_icmp
proto_tcp
proto_udp
conn_state_OTH
conn_state_REJ
conn_state_RSTO
conn_state_RSTOS0
conn_state_RSTR
conn_state_RSTRH
conn_state_S0
conn_state_S1
conn_state_S2
conn_state_S3
conn_state_SF
conn_state_SH
conn_state_SHR


In [10]:
print(df.columns.tolist())

['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']


In [11]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = df['label']

In [ ]:
start = time.time()
print('program start...')
print()

svc_gpu = SVC(C=1, kernel='linear', gamma=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)
svc_gpu.fit(X_train, Y_train)
print()

print('Prediction:')
y_pred = svc_gpu.predict(X_test)
print(y_pred)
print()

print('Score:')
score = svc_gpu.score(X_test,Y_test)
print(score)
print()

end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...



In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))